# 10. 多任务学习（Shared-Bottom / MMoE / ESMM / PLE）

工业推荐里经常是多目标：点击（CTR）、转化（CVR）、深度阅读/停留、点赞、收藏、分享……

这节我们在新闻数据上把“多任务建模”模块补齐：

- **Shared-Bottom**：共享底座 + 任务塔（基线）
- **MMoE**：多专家 + gate（常见面试题）
- **ESMM**：CTR/CVR 去偏建模（需要转化类标签）
- **PLE**：分层专家抽取（多任务更强的结构）

## 重要说明（关于标签）

公开新闻数据集通常只有点击日志，没有真实的“转化/停留/点赞”等多目标反馈。

为了让项目具备可运行的多任务训练流程，这里构造一个**可解释的 proxy 标签**：

- `ctr`：是否为验证目标点击（来自 `rank_train.pkl` 的 label）
- `ctcvr`：是否为“高价值点击”= `ctr==1` 且 `words_count >= THRESH_WORDS`

你在简历/面试里要明确写清楚：

- 这是 proxy label（用于展示方法与工程能力）
- 真实业务应使用真实转化/停留等日志，并做样本选择偏差（SSB）处理


In [6]:
import os
import pickle
from pathlib import Path
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
import tensorflow as tf
from dotenv import find_dotenv, load_dotenv
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

tf.get_logger().setLevel('ERROR')


def find_repo_root(start: Path) -> Path:
    cur = start
    for _ in range(10):
        if (cur / 'pyproject.toml').exists() or (cur / '.git').exists():
            return cur
        if cur.parent == cur:
            break
        cur = cur.parent
    return start


REPO_ROOT = find_repo_root(Path.cwd())
dotenv_path = find_dotenv(usecwd=True)
if dotenv_path:
    load_dotenv(dotenv_path)
os.environ.setdefault('FUNREC_RAW_DATA_PATH', str(REPO_ROOT / 'data'))
os.environ.setdefault('FUNREC_PROCESSED_DATA_PATH', str(REPO_ROOT / 'tmp'))

RAW_DATA_PATH = Path(os.getenv('FUNREC_RAW_DATA_PATH'))
PROCESSED_DATA_PATH = Path(os.getenv('FUNREC_PROCESSED_DATA_PATH'))
PROJECT_PATH = PROCESSED_DATA_PATH / 'projects' / 'news_recommendation_system'

ARTIFACTS_DIR = PROJECT_PATH / 'artifacts' / 'multitask'
ARTIFACTS_DIR.mkdir(parents=True, exist_ok=True)

PROJECT_PATH, ARTIFACTS_DIR


(PosixPath('/Users/wangjunfei/Desktop/fun-rec/tmp/projects/news_recommendation_system'),
 PosixPath('/Users/wangjunfei/Desktop/fun-rec/tmp/projects/news_recommendation_system/artifacts/multitask'))

In [7]:
rank_path = PROJECT_PATH / 'rank_train.pkl'
if not rank_path.exists():
    raise FileNotFoundError(f'Missing {rank_path}. Run 5.feature_engineering.ipynb first.')

df = pd.read_pickle(rank_path)

THRESH_WORDS = 300
df['ctr'] = df['label'].astype(int)
df['ctcvr'] = ((df['label'] == 1) & (df['words_count'].fillna(0) >= THRESH_WORDS)).astype(int)

# 按 user 划分训练/验证
rng = np.random.default_rng(42)
users = df['user_id'].unique()
rng.shuffle(users)
split = int(len(users) * 0.8)
train_users = set(users[:split])

train_df = df[df['user_id'].isin(train_users)].copy()
valid_df = df[~df['user_id'].isin(train_users)].copy()

DEBUG = True
MAX_TRAIN_ROWS = 300000
if DEBUG and len(train_df) > MAX_TRAIN_ROWS:
    train_df = train_df.sample(MAX_TRAIN_ROWS, random_state=42)

train_df[['ctr', 'ctcvr']].mean(), valid_df[['ctr', 'ctcvr']].mean(), len(train_df), len(valid_df)


(ctr      0.007843
 ctcvr    0.000070
 dtype: float64,
 ctr      0.007785
 ctcvr    0.000080
 dtype: float64,
 300000,
 200000)

## 1) 特征准备

和深度排序一致：

- sparse：user/item/category 等
- dense：召回分数 + 统计/时间/相似度等


In [8]:
SPARSE_COLS = ['user_id', 'article_id', 'category_id', 'user_top_category', 'last_click_article_id']
DENSE_COLS = [
    'recall_score', 'recall_rank',
    'user_click_count', 'user_unique_items',
    'item_click_count', 'words_count',
    'item_age_hours', 'time_gap_hours',
    'emb_sim_last', 'is_same_category',
]


def encode_categorical(train_s: pd.Series, valid_s: pd.Series) -> Tuple[np.ndarray, np.ndarray, int, List[str]]:
    all_s = pd.concat([train_s, valid_s], axis=0)
    codes, uniques = pd.factorize(all_s.astype(str), sort=True)
    codes = codes.astype(np.int32) + 1  # 0 reserved
    train_codes = codes[: len(train_s)]
    valid_codes = codes[len(train_s) :]
    vocab_size = int(len(uniques) + 1)
    return train_codes, valid_codes, vocab_size, list(uniques)


train_sparse = {}
valid_sparse = {}
vocab_sizes = {}
factor_uniques = {}
for col in SPARSE_COLS:
    tr, va, vs, uniq = encode_categorical(train_df[col].fillna(0), valid_df[col].fillna(0))
    train_sparse[col] = tr
    valid_sparse[col] = va
    vocab_sizes[col] = vs
    factor_uniques[col] = uniq

scaler = StandardScaler()
X_train_dense = scaler.fit_transform(train_df[DENSE_COLS].fillna(0).values.astype('float32'))
X_valid_dense = scaler.transform(valid_df[DENSE_COLS].fillna(0).values.astype('float32'))

y_train = {
    'ctr': train_df['ctr'].values.astype('float32'),
    'ctcvr': train_df['ctcvr'].values.astype('float32'),
}
y_valid = {
    'ctr': valid_df['ctr'].values.astype('float32'),
    'ctcvr': valid_df['ctcvr'].values.astype('float32'),
}

train_inputs = {**train_sparse, 'dense': X_train_dense}
valid_inputs = {**valid_sparse, 'dense': X_valid_dense}

len(y_train['ctr']), len(y_train['ctcvr'])


(300000, 300000)

In [9]:
def gauc(user_ids: np.ndarray, labels: np.ndarray, preds: np.ndarray) -> float:
    user_ids = np.asarray(user_ids)
    labels = np.asarray(labels)
    preds = np.asarray(preds)
    uniq = np.unique(user_ids)
    aucs = []
    weights = []
    for u in uniq:
        m = user_ids == u
        y = labels[m]
        if len(np.unique(y)) < 2:
            continue
        p = preds[m]
        aucs.append(roc_auc_score(y, p))
        weights.append(np.sum(m))
    if not aucs:
        return 0.0
    return float(np.average(aucs, weights=weights))


## 2) Shared-Bottom（基线）


In [10]:
def build_shared_bottom(sparse_vocab_sizes: Dict[str, int], dense_dim: int, task_names: List[str], emb_dim: int = 16, shared_units: List[int] = [128, 64], tower_units: List[int] = [64]):
    inputs = {}
    embed_vecs = []
    for feat, vocab_size in sparse_vocab_sizes.items():
        inp = tf.keras.layers.Input(shape=(), dtype=tf.int32, name=feat)
        inputs[feat] = inp
        emb = tf.keras.layers.Flatten()(tf.keras.layers.Embedding(vocab_size, emb_dim)(inp))
        embed_vecs.append(emb)
    dense_inp = tf.keras.layers.Input(shape=(dense_dim,), dtype=tf.float32, name='dense')
    inputs['dense'] = dense_inp

    x = tf.keras.layers.Concatenate()(embed_vecs + [dense_inp])
    for u in shared_units:
        x = tf.keras.layers.Dense(u, activation='relu')(x)
        x = tf.keras.layers.Dropout(0.2)(x)

    outputs = {}
    for t in task_names:
        h = x
        for u in tower_units:
            h = tf.keras.layers.Dense(u, activation='relu')(h)
        out = tf.keras.layers.Dense(1, activation='sigmoid', name=t)(h)
        outputs[t] = tf.keras.layers.Flatten()(out)

    model = tf.keras.Model(inputs=inputs, outputs=outputs, name='SharedBottom')
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-3),
        loss={t: 'binary_crossentropy' for t in task_names},
        metrics={t: [tf.keras.metrics.AUC(name='auc')] for t in task_names},
    )
    return model


shared_bottom = build_shared_bottom({k: vocab_sizes[k] for k in SPARSE_COLS}, dense_dim=X_train_dense.shape[1], task_names=['ctr', 'ctcvr'])
shared_bottom.summary()


Model: "SharedBottom"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 user_id (InputLayer)        [(None,)]                    0         []                            
                                                                                                  
 article_id (InputLayer)     [(None,)]                    0         []                            
                                                                                                  
 category_id (InputLayer)    [(None,)]                    0         []                            
                                                                                                  
 user_top_category (InputLa  [(None,)]                    0         []                            
 yer)                                                                                  

In [11]:
shared_bottom.fit(train_inputs, y_train, batch_size=2048, epochs=2, validation_data=(valid_inputs, y_valid), verbose=1)

pred = shared_bottom.predict(valid_inputs, batch_size=4096, verbose=0)
metrics_shared = {
    'ctr_auc': roc_auc_score(y_valid['ctr'], pred['ctr']),
    'ctr_gauc': gauc(valid_df['user_id'].values, y_valid['ctr'], pred['ctr']),
    'ctcvr_auc': roc_auc_score(y_valid['ctcvr'], pred['ctcvr']),
    'ctcvr_gauc': gauc(valid_df['user_id'].values, y_valid['ctcvr'], pred['ctcvr']),
}
metrics_shared


Epoch 1/2
147/147 [==============================] - 2s 8ms/step - loss: 0.1383 - flatten_13_loss: 0.0535 - flatten_12_loss: 0.0848 - flatten_13_auc: 0.4291 - flatten_12_auc: 0.6367 - val_loss: 0.0399 - val_flatten_13_loss: 0.0012 - val_flatten_12_loss: 0.0387 - val_flatten_13_auc: 0.4970 - val_flatten_12_auc: 0.8175
Epoch 2/2
147/147 [==============================] - 1s 7ms/step - loss: 0.0373 - flatten_13_loss: 9.7055e-04 - flatten_12_loss: 0.0364 - flatten_13_auc: 0.4962 - flatten_12_auc: 0.8570 - val_loss: 0.0384 - val_flatten_13_loss: 0.0011 - val_flatten_12_loss: 0.0374 - val_flatten_13_auc: 0.4956 - val_flatten_12_auc: 0.8538


{'ctr_auc': 0.86760290777641,
 'ctr_gauc': 0.8613241057502001,
 'ctcvr_auc': 0.6051488806604528,
 'ctcvr_gauc': 0.5892857142857143}

## 3) MMoE


In [12]:
def build_mmoe(
    sparse_vocab_sizes: Dict[str, int],
    dense_dim: int,
    task_names: List[str],
    emb_dim: int = 16,
    num_experts: int = 4,
    expert_units: List[int] = [128, 64],
    tower_units: List[int] = [64],
):
    inputs = {}
    embed_vecs = []
    for feat, vocab_size in sparse_vocab_sizes.items():
        inp = tf.keras.layers.Input(shape=(), dtype=tf.int32, name=feat)
        inputs[feat] = inp
        emb = tf.keras.layers.Flatten()(tf.keras.layers.Embedding(vocab_size, emb_dim)(inp))
        embed_vecs.append(emb)
    dense_inp = tf.keras.layers.Input(shape=(dense_dim,), dtype=tf.float32, name='dense')
    inputs['dense'] = dense_inp

    x = tf.keras.layers.Concatenate()(embed_vecs + [dense_inp])

    # Experts
    expert_outputs = []
    for i in range(num_experts):
        h = x
        for u in expert_units:
            h = tf.keras.layers.Dense(u, activation='relu')(h)
        expert_outputs.append(h)
    experts = tf.keras.layers.Lambda(lambda z: tf.stack(z, axis=1), name='experts_stack')(expert_outputs)  # [B, E, H]

    outputs = {}
    for t in task_names:
        gate = tf.keras.layers.Dense(num_experts, activation='softmax', name=f'gate_{t}')(x)  # [B, E]
        gate = tf.keras.layers.Lambda(lambda g: tf.expand_dims(g, axis=-1))(gate)  # [B, E, 1]
        task_inp = tf.keras.layers.Lambda(lambda z: tf.reduce_sum(z[0] * z[1], axis=1))([experts, gate])  # [B, H]

        h = task_inp
        for u in tower_units:
            h = tf.keras.layers.Dense(u, activation='relu')(h)
        out = tf.keras.layers.Dense(1, activation='sigmoid', name=t)(h)
        outputs[t] = tf.keras.layers.Flatten()(out)

    model = tf.keras.Model(inputs=inputs, outputs=outputs, name='MMoE')
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-3),
        loss={t: 'binary_crossentropy' for t in task_names},
        metrics={t: [tf.keras.metrics.AUC(name='auc')] for t in task_names},
    )
    return model


mmoe = build_mmoe({k: vocab_sizes[k] for k in SPARSE_COLS}, dense_dim=X_train_dense.shape[1], task_names=['ctr', 'ctcvr'])
mmoe.summary()


Model: "MMoE"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 user_id (InputLayer)        [(None,)]                    0         []                            
                                                                                                  
 article_id (InputLayer)     [(None,)]                    0         []                            
                                                                                                  
 category_id (InputLayer)    [(None,)]                    0         []                            
                                                                                                  
 user_top_category (InputLa  [(None,)]                    0         []                            
 yer)                                                                                          

In [13]:
mmoe.fit(train_inputs, y_train, batch_size=2048, epochs=2, validation_data=(valid_inputs, y_valid), verbose=1)

pred = mmoe.predict(valid_inputs, batch_size=4096, verbose=0)
metrics_mmoe = {
    'ctr_auc': roc_auc_score(y_valid['ctr'], pred['ctr']),
    'ctr_gauc': gauc(valid_df['user_id'].values, y_valid['ctr'], pred['ctr']),
    'ctcvr_auc': roc_auc_score(y_valid['ctcvr'], pred['ctcvr']),
    'ctcvr_gauc': gauc(valid_df['user_id'].values, y_valid['ctcvr'], pred['ctcvr']),
}
metrics_mmoe


Epoch 1/2
147/147 [==============================] - 2s 11ms/step - loss: 0.1460 - flatten_20_loss: 0.0511 - flatten_19_loss: 0.0950 - flatten_20_auc: 0.4421 - flatten_19_auc: 0.6590 - val_loss: 0.0398 - val_flatten_20_loss: 0.0012 - val_flatten_19_loss: 0.0386 - val_flatten_20_auc: 0.4995 - val_flatten_19_auc: 0.8186
Epoch 2/2
147/147 [==============================] - 2s 12ms/step - loss: 0.0360 - flatten_20_loss: 9.2743e-04 - flatten_19_loss: 0.0351 - flatten_20_auc: 0.4991 - flatten_19_auc: 0.8845 - val_loss: 0.0387 - val_flatten_20_loss: 0.0011 - val_flatten_19_loss: 0.0375 - val_flatten_20_auc: 0.5270 - val_flatten_19_auc: 0.8520


{'ctr_auc': 0.8655354882526041,
 'ctr_gauc': 0.85990851061041,
 'ctcvr_auc': 0.46000367529402353,
 'ctcvr_gauc': 0.4732142857142857}

## 4) ESMM（CTR + CVR → CTCVR）

ESMM 经典用于解决 CVR 的样本选择偏差（只在点击后才观测转化）。

这里为了能跑通，我们用 `ctcvr` proxy 标签；结构仍然与 ESMM 一致：

- 输出 `pCTR`
- 输出 `pCVR`
- 输出 `pCTCVR = pCTR * pCVR`
- 用 `ctr` 和 `ctcvr` 两个 loss 训练


In [14]:
def build_esmm(sparse_vocab_sizes: Dict[str, int], dense_dim: int, emb_dim: int = 16, shared_units: List[int] = [128, 64]):
    inputs = {}
    embed_vecs = []
    for feat, vocab_size in sparse_vocab_sizes.items():
        inp = tf.keras.layers.Input(shape=(), dtype=tf.int32, name=feat)
        inputs[feat] = inp
        emb = tf.keras.layers.Flatten()(tf.keras.layers.Embedding(vocab_size, emb_dim)(inp))
        embed_vecs.append(emb)
    dense_inp = tf.keras.layers.Input(shape=(dense_dim,), dtype=tf.float32, name='dense')
    inputs['dense'] = dense_inp

    x = tf.keras.layers.Concatenate()(embed_vecs + [dense_inp])
    for u in shared_units:
        x = tf.keras.layers.Dense(u, activation='relu')(x)
        x = tf.keras.layers.Dropout(0.2)(x)

    ctr_logit = tf.keras.layers.Dense(1, activation=None)(x)
    cvr_logit = tf.keras.layers.Dense(1, activation=None)(x)
    pctr = tf.keras.layers.Activation('sigmoid', name='ctr')(ctr_logit)
    pcvr = tf.keras.layers.Activation('sigmoid', name='cvr')(cvr_logit)
    pctcvr = tf.keras.layers.Multiply(name='ctcvr')([pctr, pcvr])

    outputs = {
        'ctr': tf.keras.layers.Flatten()(pctr),
        'ctcvr': tf.keras.layers.Flatten()(pctcvr),
    }

    model = tf.keras.Model(inputs=inputs, outputs=outputs, name='ESMM')
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-3),
        loss={'ctr': 'binary_crossentropy', 'ctcvr': 'binary_crossentropy'},
        metrics={'ctr': [tf.keras.metrics.AUC(name='auc')], 'ctcvr': [tf.keras.metrics.AUC(name='auc')]},
    )
    return model


esmm = build_esmm({k: vocab_sizes[k] for k in SPARSE_COLS}, dense_dim=X_train_dense.shape[1])
esmm.summary()


Model: "ESMM"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 user_id (InputLayer)        [(None,)]                    0         []                            
                                                                                                  
 article_id (InputLayer)     [(None,)]                    0         []                            
                                                                                                  
 category_id (InputLayer)    [(None,)]                    0         []                            
                                                                                                  
 user_top_category (InputLa  [(None,)]                    0         []                            
 yer)                                                                                          

In [15]:
esmm.fit(train_inputs, y_train, batch_size=2048, epochs=2, validation_data=(valid_inputs, y_valid), verbose=1)

pred = esmm.predict(valid_inputs, batch_size=4096, verbose=0)
metrics_esmm = {
    'ctr_auc': roc_auc_score(y_valid['ctr'], pred['ctr']),
    'ctr_gauc': gauc(valid_df['user_id'].values, y_valid['ctr'], pred['ctr']),
    'ctcvr_auc': roc_auc_score(y_valid['ctcvr'], pred['ctcvr']),
    'ctcvr_gauc': gauc(valid_df['user_id'].values, y_valid['ctcvr'], pred['ctcvr']),
}
metrics_esmm


Epoch 1/2
147/147 [==============================] - 2s 8ms/step - loss: 0.1346 - flatten_27_loss: 0.0304 - flatten_26_loss: 0.1042 - flatten_27_auc: 0.4099 - flatten_26_auc: 0.5975 - val_loss: 0.0411 - val_flatten_27_loss: 0.0017 - val_flatten_26_loss: 0.0394 - val_flatten_27_auc: 0.4851 - val_flatten_26_auc: 0.7976
Epoch 2/2
147/147 [==============================] - 1s 6ms/step - loss: 0.0386 - flatten_27_loss: 0.0015 - flatten_26_loss: 0.0371 - flatten_27_auc: 0.5308 - flatten_26_auc: 0.8438 - val_loss: 0.0388 - val_flatten_27_loss: 0.0015 - val_flatten_26_loss: 0.0373 - val_flatten_27_auc: 0.5157 - val_flatten_26_auc: 0.8373


{'ctr_auc': 0.860331500577856,
 'ctr_gauc': 0.8579555136119958,
 'ctcvr_auc': 0.4630882970637651,
 'ctcvr_gauc': 0.48341836734693877}

## 5) PLE（简化版：1 层）

PLE 比 MMoE 更常见于多任务强耦合场景；这里实现一个最小可用版本：

- shared experts + task experts
- task gate 在 shared+task experts 上做混合


In [16]:
def build_ple(
    sparse_vocab_sizes: Dict[str, int],
    dense_dim: int,
    task_names: List[str],
    emb_dim: int = 16,
    num_shared_experts: int = 2,
    num_task_experts: int = 2,
    expert_units: List[int] = [128, 64],
    tower_units: List[int] = [64],
):
    inputs = {}
    embed_vecs = []
    for feat, vocab_size in sparse_vocab_sizes.items():
        inp = tf.keras.layers.Input(shape=(), dtype=tf.int32, name=feat)
        inputs[feat] = inp
        emb = tf.keras.layers.Flatten()(tf.keras.layers.Embedding(vocab_size, emb_dim)(inp))
        embed_vecs.append(emb)
    dense_inp = tf.keras.layers.Input(shape=(dense_dim,), dtype=tf.float32, name='dense')
    inputs['dense'] = dense_inp

    x = tf.keras.layers.Concatenate()(embed_vecs + [dense_inp])

    def make_expert(name: str):
        h = x
        for u in expert_units:
            h = tf.keras.layers.Dense(u, activation='relu', name=f'{name}_dense_{u}')(h)
        return h

    shared_experts = [make_expert(f'shared_{i}') for i in range(num_shared_experts)]

    outputs = {}
    for t in task_names:
        task_experts = [make_expert(f'{t}_expert_{i}') for i in range(num_task_experts)]
        all_experts = shared_experts + task_experts
        experts_stack = tf.keras.layers.Lambda(lambda z: tf.stack(z, axis=1), name=f'{t}_experts_stack')(all_experts)

        gate = tf.keras.layers.Dense(len(all_experts), activation='softmax', name=f'{t}_gate')(x)
        gate = tf.keras.layers.Lambda(lambda g: tf.expand_dims(g, axis=-1))(gate)
        task_inp = tf.keras.layers.Lambda(lambda z: tf.reduce_sum(z[0] * z[1], axis=1))([experts_stack, gate])

        h = task_inp
        for u in tower_units:
            h = tf.keras.layers.Dense(u, activation='relu')(h)
        out = tf.keras.layers.Dense(1, activation='sigmoid', name=t)(h)
        outputs[t] = tf.keras.layers.Flatten()(out)

    model = tf.keras.Model(inputs=inputs, outputs=outputs, name='PLE_1layer')
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-3),
        loss={t: 'binary_crossentropy' for t in task_names},
        metrics={t: [tf.keras.metrics.AUC(name='auc')] for t in task_names},
    )
    return model


ple = build_ple({k: vocab_sizes[k] for k in SPARSE_COLS}, dense_dim=X_train_dense.shape[1], task_names=['ctr', 'ctcvr'])
ple.summary()


Model: "PLE_1layer"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 user_id (InputLayer)        [(None,)]                    0         []                            
                                                                                                  
 article_id (InputLayer)     [(None,)]                    0         []                            
                                                                                                  
 category_id (InputLayer)    [(None,)]                    0         []                            
                                                                                                  
 user_top_category (InputLa  [(None,)]                    0         []                            
 yer)                                                                                    

In [17]:
ple.fit(train_inputs, y_train, batch_size=2048, epochs=2, validation_data=(valid_inputs, y_valid), verbose=1)

pred = ple.predict(valid_inputs, batch_size=4096, verbose=0)
metrics_ple = {
    'ctr_auc': roc_auc_score(y_valid['ctr'], pred['ctr']),
    'ctr_gauc': gauc(valid_df['user_id'].values, y_valid['ctr'], pred['ctr']),
    'ctcvr_auc': roc_auc_score(y_valid['ctcvr'], pred['ctcvr']),
    'ctcvr_gauc': gauc(valid_df['user_id'].values, y_valid['ctcvr'], pred['ctcvr']),
}
metrics_ple


Epoch 1/2
147/147 [==============================] - 2s 13ms/step - loss: 0.1369 - flatten_34_loss: 0.0591 - flatten_33_loss: 0.0778 - flatten_34_auc: 0.5058 - flatten_33_auc: 0.6765 - val_loss: 0.0394 - val_flatten_34_loss: 0.0011 - val_flatten_33_loss: 0.0382 - val_flatten_34_auc: 0.5000 - val_flatten_33_auc: 0.8251
Epoch 2/2
147/147 [==============================] - 2s 15ms/step - loss: 0.0358 - flatten_34_loss: 8.9627e-04 - flatten_33_loss: 0.0349 - flatten_34_auc: 0.5000 - flatten_33_auc: 0.8841 - val_loss: 0.0390 - val_flatten_34_loss: 0.0010 - val_flatten_33_loss: 0.0380 - val_flatten_34_auc: 0.4999 - val_flatten_33_auc: 0.8606


{'ctr_auc': 0.8709192230428464,
 'ctr_gauc': 0.863565464721534,
 'ctcvr_auc': 0.4109231863549084,
 'ctcvr_gauc': 0.43112244897959184}

## 6) 总结与保存


In [18]:
summary = pd.DataFrame([
    {'model': 'shared_bottom', **metrics_shared},
    {'model': 'mmoe', **metrics_mmoe},
    {'model': 'esmm', **metrics_esmm},
    {'model': 'ple', **metrics_ple},
])
summary


,model,ctr_auc,ctr_gauc,ctcvr_auc,ctcvr_gauc
0,shared_bottom,0.867603,0.861324,0.605149,0.589286
1,mmoe,0.865535,0.859909,0.460004,0.473214
2,esmm,0.860332,0.857956,0.463088,0.483418
3,ple,0.870919,0.863565,0.410923,0.431122


In [19]:
shared_bottom.save(ARTIFACTS_DIR / 'shared_bottom.keras')
mmoe.save(ARTIFACTS_DIR / 'mmoe.keras')
esmm.save(ARTIFACTS_DIR / 'esmm.keras')
ple.save(ARTIFACTS_DIR / 'ple.keras')

with open(ARTIFACTS_DIR / 'scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
with open(ARTIFACTS_DIR / 'vocab_sizes.pkl', 'wb') as f:
    pickle.dump(vocab_sizes, f)
summary.to_csv(ARTIFACTS_DIR / 'multi_task_report.csv', index=False)

print('saved to:', ARTIFACTS_DIR)


saved to: /Users/wangjunfei/Desktop/fun-rec/tmp/projects/news_recommendation_system/artifacts/multitask
